# The BatAnalysis Survey Data Analysis Code

### This notebook will go through how the BatAnalysis code can be used to analyze BAT survey data.

In [1]:
import batanalysis as ba
import matplotlib.pyplot as plt
%matplotlib notebook
import os
os.environ['HEADASNOQUERY']='1' #This line is for dealing with heasoftpy outputs in jupyter notebooks

import numpy as np

#we will be querying from HEASARC so we will import this from astroquery
from astroquery.heasarc import Heasarc
heasarc=Heasarc()


##############.   Downloading the data at our preferred location...

object_name='J221951-48424'
mission = 'swiftmastr'
table = heasarc.query_object(object_name, mission=mission, sortvar="START_TIME")
obs_dir="/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/"
#table.pprint_all()
#print(table[:5])
ba.download_swiftdata(table[:5], save_dir="/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data") 
#Or esle it will save it in some place else: ~/.swift/swift_data_dir/




{'03108111001': {'success': True,
  'obsoutdir': PosixPath('/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/03108111001'),
  'quicklook': False,
  'data': Swift_Data(username='anonymous',obsid='03108111001',quicklook='False',auxil='True',bat='True',xrt='False',uvot='False',log='False')},
 '03108111002': {'success': True,
  'obsoutdir': PosixPath('/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/03108111002'),
  'quicklook': False,
  'data': Swift_Data(username='anonymous',obsid='03108111002',quicklook='False',auxil='True',bat='True',xrt='False',uvot='False',log='False')},
 '07021582001': {'success': True,
  'obsoutdir': PosixPath('/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/07021582001'),
  'quicklook': False,
  'data': Swift_Data(username='anonymous',obsid='07021582001',quicklook='False',auxil='True',bat='True',xrt='False',uvot='False',log='False')},
 '07400048001': {'s

In [5]:

################.  BAT Survey
batsurvey_obs=[]
obs_dir="/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/"
#input_dict=dict(obs_dir)       #Creating a dictionary to set up the input and output directories and the files

#FRB180916_dir="/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data"

for i in table["OBSID"][:]:

    print(i)
    try:
        
        obs=ba.BatSurvey(str(i),recalc=True,obs_dir="/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/")
    
        obs.save()
        batsurvey_obs.append(obs)
    except ValueError:
        print("Obsid has no survey data")




03108111001
A save file has been written to /Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/03108111001_surveyresult/batsurvey.pickle.
03108111002
Obsid has no survey data
07021582001
A save file has been written to /Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/07021582001_surveyresult/batsurvey.pickle.
Obsid has no survey data
07400048001
A save file has been written to /Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/07400048001_surveyresult/batsurvey.pickle.
07400048002
A save file has been written to /Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data/07400048002_surveyresult/batsurvey.pickle.
07400048003


FileNotFoundError: The directory /Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/J221951_48424_data does not contain the observation data corresponding to ID: 07400048003

In [3]:
source_name="J221951-48424"
source_id="J221951-48424"
for obs in batsurvey_obs:

    print("Running calculate_PHA for observation id", obs.pointing_ids)
    
    obs.merge_pointings()
    obs.calculate_pha(id_list=source_id, clean_dir=True)
    pha_list=obs.get_pha_filenames(id_list=source_name)
    ba.calc_response(pha_list)
    
    for pha in pha_list: 
        try:
            ba.fit_spectrum(pha, obs)
            print(pha)
        except Exception as Error_with_Xspec_fitting:
            #If XSPEC cannot fit because of negative counts
            print (Error_with_Xspec_fitting)
            #the fitting wasnt not successful and the dictionary was not created but want to enter the upper limit if
            #statement
            fluxerr_lolim=0
            flux=1
            nsigma=1
            avg_flux_err=1
            
            
 
        

Running calculate_PHA for observation id ['20200371529']
batsurvey-catmux v0.1
----------------------------------------------------------
Creating PHA file for  FRB180916
Time interval: [602695795.0] [602696883.0]

1 spectrum  in use
 
Spectral Data File: /Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005_surveyresult/PHA_files/FRB180916_survey_point_20200371529.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  -2.134e-04 +/- 3.770e-04
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-8
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1088 sec
 Using fit statistic: chi
Error executing command: fit
 Using Response (RMF) File            FRB180916_survey_point_20200371529.rsp for Source 1


Model cflux<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   cflux      Emin       keV      0.500000     frozen
   2    1   cflux      Emax   

***Warning: POISSERR keyword is missing or of wrong format, assuming FALSE.

***XSPEC Error:  No variable parameters for fit 

'\n # Capturing the Flux and its error. saved to the model object, can be obtained by calling model(1).error, model(2).error\n            model_params=dict()\n            for i in range(1,model.nParameters+1):\n                xsp.Fit.error("2.706 %d"%(i))\n\n            #get the name of the parameter\n                par_name=model(i).name\n                model_params[par_name]=dict(val=model(i).values[0], lolim=model(i).error[0], hilim=model(i).error[1], errflag=model(i).error[-1])\n            surveyobservation.set_pointing_info(pointing_id,"model_params", model_params, source_id=source_id)\n\n    #Incorporating the model names, parameters, errors into the BatSurvey object.\n            xsp.Xset.save(pha_file.split(".")[0])\n            xspec_savefile=os.path.join(pha_dir, pha_file.split(".")[0]+".xcm")\n            surveyobservation.set_pointing_info(pointing_id,"xspec_model",xspec_savefile,source_id=source_id)\n\n    #ba.calculate_detection(obs, source_name)\n    \n    \n  '

In [4]:
#Loop over individual PHA pointings

    
    
    
    
   
     

In [5]:
     
#ba.print_parameters(batsurvey_obs,source_name, values=["met_time","utc_time", "exposure","lg10Flux", "PhoIndex"],savetable=True,save_file="out.txt")
    



In [6]:
source_id="FRB180916"

ba.print_parameters(batsurvey_obs,source_id, values=["met_time","utc_time", "exposure","lg10Flux", "PhoIndex"],savetable=True,save_file="/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/notebooks/out.txt")
    



In [7]:
pointing_ids=obs.get_pointing_ids()
print(pointing_ids)

pointing_dict=obs.get_pointing_info(pointing_ids[0])
print(pointing_dict)

source_dict=obs.get_pointing_info(pointing_ids[0], source_id=source_name)
print(source_dict)

['20200371529']
{'met_time': 602695795.0, 'exposure': 1088.0, 'utc_time': numpy.datetime64('2020-02-06T15:29:30.610148809'), 'mjd_time': 58885.64549317302, 'FRB180916': {'model_params': {'Emin': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'Emax': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'lg10Flux': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'PhoIndex': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'norm': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}}, 'xspec_model': '/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005_surveyresult/PHA_files/FRB180916_survey_point_20200371529.xcm'}}
{'model_params': {'Emin': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'Emax': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'lg10Flux': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'Ph

In [8]:
print(batsurvey_obs[0].get_pointing_info("20200371529"))

{'met_time': 602695795.0, 'exposure': 1088.0, 'utc_time': numpy.datetime64('2020-02-06T15:29:30.610148809'), 'mjd_time': 58885.64549317302, 'FRB180916': {'model_params': {'Emin': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'Emax': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'lg10Flux': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'PhoIndex': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'norm': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}}, 'xspec_model': '/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005_surveyresult/PHA_files/FRB180916_survey_point_20200371529.xcm'}}


In [10]:
print(batsurvey_obs[0].get_pointing_info("20200371529"))

{'met_time': 602695795.0, 'exposure': 1088.0, 'utc_time': numpy.datetime64('2020-02-06T15:29:30.610148809'), 'mjd_time': 58885.64549317302, 'FRB180916': {'model_params': {'Emin': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'Emax': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'lg10Flux': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'PhoIndex': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}, 'norm': {'val': nan, 'lolim': nan, 'hilim': nan, 'errflag': 'TTTTTTTTT'}}, 'xspec_model': '/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005_surveyresult/PHA_files/FRB180916_survey_point_20200371529.xcm'}}


In [11]:
ba.calculate_detection(obs, source_name)


The condition here is nan 3 nan nan
No detection, just upperlimits for the spectrum: FRB180916_survey_point_20200371529.pha
Creating PHA file for  FRB180916
Time interval: [602695795.0] [602696883.0]

1 spectrum  in use
 
Spectral Data File: FRB180916_survey_point_20200371529_bkgnsigma_5_upperlim.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.223e-03 +/- 5.308e-04
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-8
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1088 sec
 Using fit statistic: cstat
 Using Response (RMF) File            FRB180916_survey_point_20200371529_bkgnsigma_5_upperlim.rsp for Source 1

******************************************************
Fitting the 5 times bkg of the spectrum  FRB180916_survey_point_20200371529_bkgnsigma_5_upperlim.pha
******************************************************
******************************************************
******************************************************
(3.842548631203

***Warning: POISSERR keyword is missing or of wrong format, assuming FALSE.


[]


Fit statistic  : C-Statistic                   63.89     using 8 bins.

    Source file is not Poisson for spectrum 1


Test statistic : Chi-Squared                  238.70     using 8 bins.
 Null hypothesis probability of 1.06e-48 with 6 degrees of freedom
 Current data and model not fit yet.

Model powerlaw<1> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
________________________________________________________________________


Fit statistic  : C-Statistic                  645.40     using 8 bins.

    Source file is not Poisson for spectrum 1


Test statistic : Chi-Squared              1.891658e+06     using 8 bins.
 Null hypothesis probability of 0.000000e+00 with 6 degrees of freedom
 Current data and model not fit yet.

Fit statistic  : C-Statistic                  645.40     using 8 bins.



In [12]:
ba.print_parameters(obs,source_id, values=["met_time","utc_time", "exposure","lg10Flux", "PhoIndex"],savetable=False,save_file="/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/notebooks/out.txt")
    



Obs ID  	 Pointing ID		met_time	utc_time	exposure	lg10Flux	PhoIndex
00013201005	20200371529	602695795.0	2020-02-06T15:29:30.610148809	1088.0	  3.842549e-09  	nan-nan\+nan
           


In [13]:
print(obs.__dict__)

{'channel': [1, 2, 3, 4, 5, 6, 7, 8], 'emin': [14.0, 20.0, 24.0, 35.0, 50.0, 75.0, 100.0, 150.0], 'emax': [20.0, 24.0, 35.0, 50.0, 75.0, 100.0, 150.0, 195.0], 'syserr': [0.6, 0.3, 0.15, 0.15, 0.15, 0.15, 0.15, 0.6], 'pha_file_names_list': ['/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005_surveyresult/PHA_files/FRB180916_survey_point_20200371529.pha', '/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005_surveyresult/PHA_files/FRB180916_survey_point_20200371529_bkgnsigma_5_upperlim.pha'], 'obs_id': '00013201005', 'obs_dir': '/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005', 'survey_input': {'indir': '/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005', 'outdir': '/Users/slaha/Desktop/BAT_projects/GIT_Repository_directory_Jan2022/survey_data/FRB180916_data/00013201005